In [1]:
!pip install numpy 

In [2]:
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
!pip install scikit-learn 

In [3]:
!pip uninstall numpy --yes 
!pip install numpy==1.26.0 

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
  Using cached numpy-1.26.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (53 kB)
Using cached numpy-1.26.0-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tableone 0.9.1 requires openpyxl>=3.1.2, which is not installed.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [5]:
filename = "/Users/jocelyn/Desktop/DXC-4 Technology /AI-Enhanced-Customer-Interaction-Assistant/data/Final_db.csv" 
df = pd.read_csv(filename, header=0) 

In [6]:
print(df.head()) 
print(df.columns) 

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
2  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
3  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
4  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
2    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
3    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
4    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2017-10-04 19:55:00           2017-10

In [7]:
# check to see if there are products with multiple orders 
product_id_value_counts = df["product_id"].value_counts() 
print(product_id_value_counts) 

# store the products with multiple orders in products_multiple_orders 
products_multiple_orders = product_id_value_counts[product_id_value_counts > 1] 

product_id
aca2eb7d00ea1a7b8ebd4e68314663af    536
99a4788cb24856965c36a24e339b6058    528
422879e10f46682990de24d770e7f83d    508
389d119b48cf3043d311335e499d9c6b    406
368c6c730842d78016ad823897a372db    398
                                   ... 
ef34878717e7759a53c99550100ea1ec      1
6965419e0d5116b685c3f825e4752231      1
a2d0ef58f86a42fa9d456709cb00c0b2      1
4e1d8ab59a756d83db0a0902c1342e11      1
006619bbed68b000c8ba3f8725d5409e      1
Name: count, Length: 32951, dtype: int64


In [8]:
# create a new column in df that stores the number of times each item was purchased 
df["times_purchased"] = df["product_id"].map(product_id_value_counts) 

# check to see if the column is correctly added in 
print(df[["product_id", "times_purchased"]]) 
print(df[df["product_id"] == "aca2eb7d00ea1a7b8ebd4e68314663af"]) 

                              product_id  times_purchased
0       87285b34884572647811a353c7ac498a              6.0
1       87285b34884572647811a353c7ac498a              6.0
2       87285b34884572647811a353c7ac498a              6.0
3       595fac2a385ac33a80bd5114aec74eb8            106.0
4       aa4383b373c6aca5d8797843e5594415              3.0
...                                  ...              ...
119138  f1d4ce8c6dd66c47bbaa8c6781c2a923             19.0
119139  b80910977a37536adeddd63663f916ad              3.0
119140  d1c427060a0f73f6b889a5c7c61f2ac4            357.0
119141  d1c427060a0f73f6b889a5c7c61f2ac4            357.0
119142  006619bbed68b000c8ba3f8725d5409e              1.0

[119143 rows x 2 columns]
                                order_id                       customer_id  \
54      25f4376934e13d3508486352e11a5db0  12fd2740039676063a874b9567dfa651   
238     6d7de866a140b19d09e825b2a4e944c7  9e5ce657315b2bdb94033a494041ac25   
239     6d7de866a140b19d09e825b2a4e944c7  9

In [9]:
# items purchased multiple times 
# average rating 

# summary of all reviews for that product 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [11]:
# stores all of the unique product categories into the array "product_categories" 
product_categories = df["product_cat_name_translated"].unique() 

""" 
# output product_categories 
print(product_categories) 
""" 

""" 
# check if there are any rows where the product review titles are filled out, and the product review messages aren't 
# result --> there are 
review_titles_with_no_reviews = df[df["review_comment_title"].notna() & df["review_comment_message"].isna()] 
print("________________") 
print(review_titles_with_no_reviews["review_comment_message"]) 
""" 

# fill in rows in df["review_comment_title"] column that have na values with "" 
df["review_comment_title"] = df["review_comment_title"].fillna("") 

# fill in rows in df["review_comment_message"] column that have na values with "" 
df["review_comment_message"] = df["review_comment_message"].fillna("") 

# merge the product reviews and product review titles with a space in between both 
df["merged_review_messages_and_titles"] = df["review_comment_message"] + " " + df["review_comment_title"] 
df["merged_review_messages_and_titles"] = df["merged_review_messages_and_titles"].str.strip() 

# dictionary to store dataframes of the product reviews for all products in each category 
# products are separated by category 
product_reviews_by_category = {}; 

# fill in dictionary with keys (df category) and values (product_id and product reviews for all products in that category) 
for category in product_categories: 
    product_reviews_by_category["df_" + str(category)] = df[df["product_cat_name_translated"] == category][["product_id", "merged_review_messages_and_titles"]] 

# output product_reviews_by_category 
#for key, value in product_reviews_by_category.items(): 
#    print("\n") 
    # product category 
#    print(key + "\n") 
    # product reviews 
#    print(value) 

In [12]:
""" 
TF-IDF to distinguish between different products in same category 
""" 

# dictionary that contains the produced matrix for all product categories 
resulting_matrices = {} 

# every row (product review) in the df is treated as a document 
for keys, values in product_reviews_by_category.items(): 
    print(f"TfidfVectorizer for {keys}") 
    print("--------------------------------------------------------------------------------------") 

    # use bigrams (for cases of negation like "not good") and trigrams (for cases with numbers like "bought 3 pairs") 
    vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 

    # fit vectorizer to the product review titles and messages 
    vectorizer.fit(values["merged_review_messages_and_titles"]) 

#    print("Vocabulary size {0}: {1}\n".format(len(vectorizer.vocabulary_), vectorizer.vocabulary_.items())) 

#    print("Matrix:\n") 
    # result_matrix contains each document's TF_IDF scores 
    resulting_matrix = vectorizer.transform(values["merged_review_messages_and_titles"]) 

    print("**************************************************************************************") 
#    print(vectorizer.get_feature_names_out()) 
    
    product_id = values["product_id"].tolist() 
    
    resulting_matrix = pd.DataFrame(resulting_matrix.toarray(), index=product_id, columns=vectorizer.get_feature_names_out()) 
    
    # resulting_matrices is a dictionary containing the resulting_matrix for all product categories 
    key_name = f"resulting_matrix_{keys}" 
    resulting_matrices[key_name] = resulting_matrix 
    
#    print(resulting_matrix.todense()) 

#    print("\nHeatmap of Matrix:\n") 
#    df_print = pd.DataFrame(resulting_matrix.toarray(), columns=vectorizer.get_feature_names_out()) 
#   plt.rcParams['figure.figsize'] = [11, 11] 
#    ax = sns.heatmap(df_print, annot=True, cmap='coolwarm', cbar=False, 
#                     yticklabels=[f"Document {i+1}:" for i in range(len(values))]) 
#    _ =ax.set_title(f'TF-IDF Matrix {keys}');
#    _ =ax.set_yticklabels(ax.get_yticklabels(), rotation = 0, fontsize = 15) 

TfidfVectorizer for df_housewares
--------------------------------------------------------------------------------------
**************************************************************************************
TfidfVectorizer for df_perfumery
--------------------------------------------------------------------------------------
**************************************************************************************
TfidfVectorizer for df_auto
--------------------------------------------------------------------------------------
**************************************************************************************
TfidfVectorizer for df_pet_shop
--------------------------------------------------------------------------------------
**************************************************************************************
TfidfVectorizer for df_stationery
--------------------------------------------------------------------------------------
********************************************************

In [13]:
# check to see if resulting_matrices contains the various matrices with product_id as index and individual columns for each n-gram 
#for key, values in resulting_matrices.items(): 
#    print(key) 
#    print(values) 

In [14]:
# replace document number with product_id, and replace term index with the actual term 

In [15]:
# analyze inputted user response and determine which product best matches it 
# look at distinguishing features across categories 
# ask user "is this good?" 
# if yes, then also output other related recommendations 

In [16]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score 

In [18]:
!pip install gensim 

In [19]:
import gensim 

In [20]:
# sample_user_input 
sample_user_input = "Recommend me a sturdy chair" 

# preprocess user input ~ "this lowercases [and] tokenizes"; stop words get removed 
sample_user_input = list(gensim.utils.simple_preprocess(sample_user_input)) 
print(sample_user_input) 

['recommend', 'me', 'sturdy', 'chair']


In [21]:
# feature ~ product title and review 
X = df["merged_review_messages_and_titles"] 

# label ~ product category 
y = df["product_cat_name_translated"] 

# from sklearn.feature_extraction.text import CountVectorizer 

# vectorizer = CountVectorizer(ngram_range=(2, 3)) 
# X_n = vectorizer.fit_transform(df["merged_review_messages_and_titles"]) 
# product_id = df["product_id"].tolist() 
    
# resulting_matrix = pd.DataFrame(X_n.toarray(), index=product_id, columns=vectorizer.get_feature_names_out())     
# print(resulting_matrix) 

In [22]:
# preprocess each (each row) product title and review 
X = X.apply(lambda row: gensim.utils.simple_preprocess(row)) 

In [23]:
# each row in X is now tokenized 
print(X) 

0         [não, testei, produto, ainda, mas, ele, veio, ...
1         [não, testei, produto, ainda, mas, ele, veio, ...
2         [não, testei, produto, ainda, mas, ele, veio, ...
3                   [muito, bom, produto, muito, boa, loja]
4                                                        []
                                ...                        
119138    [so, uma, peça, que, veio, rachado, mas, tudo,...
119139                    [foi, entregue, antes, do, prazo]
119140    [foi, entregue, somente, quero, saber, do, out...
119141    [foi, entregue, somente, quero, saber, do, out...
119142                                                   []
Name: merged_review_messages_and_titles, Length: 119143, dtype: object


In [24]:
# split data into a training set and a test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=1111) 

In [37]:
print("Begin") 
word2vec_model = gensim.models.Word2Vec(X_train, vector_size=100, window=5, min_count=1) 
print("End") 
print(len(word2vec_model.wv.key_to_index)) 

Begin
End
14005


In [38]:
# linear regression to determine which category the user input product likely belongs to 
# look in the category and use word vectors to determine which product is most similar to 
# the one that the user is looking for 

In [39]:
# size of word2vec_model 
print(len(word2vec_model.wv.key_to_index)) 

14005


In [40]:
print(word2vec_model.wv.most_similar("saber")) 

[('gostaria', 0.9232568740844727), ('receber', 0.8663808107376099), ('vão', 0.8646805286407471), ('vai', 0.8601102828979492), ('devolver', 0.8566531538963318), ('verificar', 0.8530604839324951), ('responder', 0.8529185652732849), ('querer', 0.8502118587493896), ('mandar', 0.8437370657920837), ('preciso', 0.8415015935897827)]


In [47]:
# replace every word in the dataset that's also found in word2vec_model with their word embedding 
# a set with all of the unique words in the word2vex_model 
words = set(word2vec_model.wv.index_to_key) 

# X_train word embeddings 
print("Begin Transforming X_train") 
# array for all of the word embeddings for X_train 
X_train_word_embeddings = []; 
for row in X_train: 
    row_embedding = [] 
    for word in words: 
        if word in row: 
            row_embedding.append(word2vec_model.wv[word]) 
    X_train_word_embeddings.append(np.array(row_embedding)) 
print("Finish Transforming X_train") 

# X_test word embeddings 
print("Begin Transforming X_test") 
# array for all of the word embeddings for X_test 
X_test_word_embeddings = []; 
for row in X_test: 
    row_embedding = []; 
    for word in words: 
        if word in row: 
            row_embedding.append(word2vec_model.wv[word]) 
    X_test_word_embeddings.append(np.array(row_embedding)) 
print("Finish Transforming X_test") 

Begin Transforming X_train
Finish Transforming X_train
Begin Transforming X_test
Finish Transforming X_test


In [48]:
X_train_feature_vector = []
for w in X_train_word_embeddings:
    if w.size:
        X_train_feature_vector.append(w.mean(axis=0))
    else:
        X_train_feature_vector.append(np.zeros(100, dtype=float))
        
X_test_feature_vector = []
for w in X_test_word_embeddings:
    if w.size:
        X_test_feature_vector.append(w.mean(axis=0))
    else:
        X_test_feature_vector.append(np.zeros(100, dtype=float))

print(X_train_feature_vector[1:11]) 

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([-0.14534935,  0.27452433, -0.2145524 , -0.02878416, -0.10788985,
       -0.6340307 ,  0.10151728,  1.011183  , -0.8562664 , -0.88614243,
       -0.11211656, -0.24360485,  0.21715567,  0.00991838,  0.06129951,
       -0.20033026,  0.46333212, -0.30022234, -0.33242127, -0.80321395,
        0.2852601 , -0.41565737,  0.6484374 , -0.04431574, -0.23141341,
        0.39273864, -0.53634727, -0.3802703 , -0.32483652,  0.22657308,
        0.2649581 ,  0.09945676, -0.03661419, -0.48914558, -0.1765134 ,
        0.28487983, -0.2973519 , -0.19904219, -0.05

In [62]:
# logistic regression 
model = LogisticRegression(max_iter=500)
model.fit(X_train_feature_vector, y_train)

probability_predictions = model.predict_proba(X_test_feature_vector) 

# predict most likely product category 
class_label_predictions = model.predict(X_test_feature_vector) 

In [ ]:
#sample_user_input_list = list(gensim.utils.simple_preprocess(sample_user_input)) 
#bigram_transformer = Phrases([sample_user_input_list]) 
#aa = list(bigram_transformer[sample_user_input_list]) 
#model = gensim.models.Word2Vec([aa], vector_size=200, window=5, min_count=1) 
#print(len(model.wv.key_to_index)) 
#print(model.wv.most_similar("me")) 

In [195]:
user_input = input("Hi! What product are you looking for?") 

#user_input_word2vec_model = gensim.models.Word2Vec(user_input, vector_size=200, window=5, min_count=1) 

# preprocess user input ~ "this lowercases [and] tokenizes"; stop words get removed 
user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 

# create a list containing the vectors for the tokens/words found in the user input that are also found in the dataset of available products 
token_vectors = [] 

while len(token_vectors) == 0: 
    user_input_tokens = list(gensim.utils.simple_preprocess(user_input)) 
    token_vectors = [] 
    
    # loop through all tokens from the tokenized version of the user input 
    for token in user_input_tokens: 
        # if the token is also found in the dataset 
        if token in word2vec_model.wv.key_to_index: 
            # add its word vector to the "token_vectors" list 
            token_vectors.append(word2vec_model.wv[token]) 

    # if there are vectors 
    if len(token_vectors) > 0: 
        # calculate the mean for each token_vector and combine into an array 
        overall_token_vector = np.mean(token_vectors, axis=0) 
        # reshape into a two dimensional array 
        overall_token_vector = token_vector.reshape(1, -1) 
        word2vec_prediction = model.predict(overall_token_vector) 
        print(word2vec_prediction) 
    
    else: 
        user_input = input("We can't find a related product. Please input more detail.") 

Hi! What product are you looking for? jio
We can't find a related product. Please input more detail. saber


['health_beauty']


In [109]:
word2vec_prediction_str = "df_" + str(word2vec_prediction[0]) 

#vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 

product_reviews_by_category[word2vec_prediction_str] 

# use bigrams (for cases of negation like "not good") and trigrams (for cases with numbers like "bought 3 pairs") 
vectorizer = TfidfVectorizer(ngram_range=(2, 3)) 

# fit vectorizer to the product review titles and messages 
vectorizer.fit(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 

# result_matrix contains each document's TF_IDF scores 
resulting_matrix = vectorizer.transform(product_reviews_by_category[word2vec_prediction_str]["merged_review_messages_and_titles"]) 
    
product_id = product_reviews_by_category[word2vec_prediction_str]["product_id"].tolist() 
    
resulting_matrix = pd.DataFrame(resulting_matrix.toarray(), index=product_id, columns=vectorizer.get_feature_names_out()) 

In [140]:
# determine product in category that's most similar to user input 
user_input_TF_IDF_vector = vectorizer.transform([user_input]) 
# similarity of user input word vector and each of the rows' word vectors 
similarity = cosine_similarity(user_input_TF_IDF_Vector, resulting_matrix) 
greatest_similarity_indices = np.argmax(similarity) 

# find index of the three row (if applicable) whose word vector shares the greatest 
# similarity with user input word vector 
#lowest_to_highest_similarity_indices = np.argsort(similarity) 

#if (len(lowest_to_highest_similarity_indices) == 3) | (len(lowest_to_highest_similarity_indices) > 3): 
#    greatest_similarity_indices = lowest_to_highest_similarity_indices[-3:] 
#    # sort the indices to be highest to lowest of the three 
#    #greatest_similarity_indices = greatest_similarity_indices[::-1] 
#elif len(lowest_to_highest_similarity_indices) == 2: 
#    greatest_similarity_indices = lowest_to_highest_similarity_indices[-2:]
#    print(lowest_to_highest_similarity_indices[-2]) 
#    # sort the indices to be highest to lowest of the three 
#    #greatest_similarity_indices = greatest_similarity_indices[::-1] 
#elif len(lowest_to_highest_similarity_indices) == 1: 
#    greatest_similarity_indices = lowest_to_highest_similarity_indices[-1:] 
#else: 
#    greatest_similarity_indices = -1 
# product_ids of the three rows whose word vector share the greatest simiilarity with user 
# input word vector 

#print(greatest_similarity_indices) 
greatest_similarity_product_ids = resulting_matrix.index[greatest_similarity_indices] 
print(greatest_similarity_product_ids) 

# out of the similar products, find the one with the highest rating 

5ac9d9e379c606e36a8094a6046f75dc


In [174]:
# determine product in category that's most similar to user input 
user_input_TF_IDF_vector = vectorizer.transform([user_input]) 
# similarity of user input word vector and each of the rows' word vectors 
similarity = cosine_similarity(user_input_TF_IDF_vector, resulting_matrix) 
greatest_similarity_indices = np.argsort(similarity[0][::-1]) 
print(greatest_similarity_indices) 
greatest_similarity_product_ids = resulting_matrix.index[greatest_similarity_indices] 
print(greatest_similarity_product_ids) 

[    0  6684  6685 ...  3347  3340 10031]
Index(['5ac9d9e379c606e36a8094a6046f75dc', 'e0cf79767c5b016251fe139915c59a26',
       '0a9a9c25c5a06532d1766e00db833a7f', '2948658cb6abc82847412be7201bfc4c',
       '921d31a1daa51460b7a95ea5f3ab64d5', '9ea1152d6d52dc57ab2ea49aa626adc1',
       '67bd616e1ba0d3d3e8545f3113b0140d', '417e213344336988e562ccc3975987f4',
       'cf2854dbad42c9f15b2e69add9bcb1a0', '719d571299707561c34ba04ab867b32a',
       ...
       '58e14150c9316b8f981742498dd97606', 'af91083bde899b201798b52ae89babea',
       'cfd414b4463647f58c7775eaae06893d', '82fd0e5d33b58ad696f75449679c1470',
       '76ad24d8518857df0f3dc45b2f5faf2d', 'adf591c625cb265c12bc6749d3a2f757',
       '7b6669ccb9510397f7923a979c895733', '03552ec45033c8658713c541f6af9f05',
       '4100a608bd0aa524b06ffefba973707b', '006619bbed68b000c8ba3f8725d5409e'],
      dtype='object', length=10032)


In [177]:
# rows with that product id 
products_df = df[df["product_id"] == greatest_similarity_product_ids[0]] 

print("How about this product?") 
print("Product: ", greatest_similarity_product_ids[0]) 
print("Price: ", products_df["price"].iloc[0]) 
print("Average Product Rating: ", products_df["review_score"].mean()) 

How about this product
Product:  5ac9d9e379c606e36a8094a6046f75dc
Price:  109.9
Average Product Rating:  1.3333333333333333


In [188]:
more_detail_user_input = input("Do you want more detail about this product? (Y/N)") 

more_detail_user_input = more_detail_user_input.upper().strip() 

more_recommendations = "N" 
more_detail = "" 

if more_detail_user_input == "Y": 
    print("Ok. Here is more detail about the product.") 
    print("Total Orders: ", len(products_df)) 
    print("Product Weight (g): ", products_df["product_weight_g"].iloc[0]) 
    print("Product Length (cm): ", products_df["product_length_cm"].iloc[0]) 
    print("Product Height (cm): ", products_df["product_height_cm"].iloc[0]) 
    print("Product Width (cm): ", products_df["product_width_cm"].iloc[0]) 
else: 
    more_recommendations = input("Would you like more similar product recommendations? (Y/N)") 

more_recommendations = more_recommendations.upper().strip() 

if more_recommendations == "Y": 
    print("Ok. Here are some more product recommendations.") 
    if (len(greatest_similarity_product_ids) == 3) | (len(greatest_similarity_product_ids) > 3): 
        products_df = df[df["product_id"] == greatest_similarity_product_ids[1]] 
    
        print("Product: ", greatest_similarity_product_ids[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 

        products_df = df[df["product_id"] == greatest_similarity_product_ids[1]] 
        print("Product: ", greatest_similarity_product_ids[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 

        products_df = df[df["product_id"] == greatest_similarity_product_ids[2]] 
        print("Product: ", greatest_similarity_product_ids[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean()) 

    elif len(greatest_similarity_product_ids) == 2: 
        products_df = df[df["product_id"] == greatest_similarity_product_ids[1]] 
    
        print("Product: ", greatest_similarity_product_ids[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean(), "\n") 

        products_df = df[df["product_id"] == greatest_similarity_product_ids[1]] 
        print("Product: ", greatest_similarity_product_ids[0]) 
        print("Price: ", products_df["price"].iloc[0]) 
        print("Average Product Rating: ", products_df["review_score"].mean()) 

    else: 
        more_detail = input("There are no more similar products. Please input more detail to get better product recommendations.") 

Do you want more detail about this product? (Y/N) n
Would you like more similar product recommendations? (Y/N) y


Ok. Here are some more product recommendations.
Product:  5ac9d9e379c606e36a8094a6046f75dc
Price:  29.9
Average Product Rating:  4.474820143884892 

Product:  5ac9d9e379c606e36a8094a6046f75dc
Price:  29.9
Average Product Rating:  4.474820143884892 

Product:  5ac9d9e379c606e36a8094a6046f75dc
Price:  14.99
Average Product Rating:  3.933333333333333


In [191]:
#recommendation_quality = input("Was that a good recommendation? (Y/N)") 

# recommend more related products 

In [110]:
# category prediction using TF-IDF? 

In [75]:
# a dictionary with a number associated with each product category 
product_categories_dict = {} 

# array with unique product categories 
product_categories = df["product_cat_name_translated"].unique() 

for i in range(len(product_categories)): 
    product_categories_dict[i+1] = product_categories[i] 

# ask user for product category 
print("Which category is the product in? Enter in number.") 
for keys, values in product_categories_dict.items(): 
    print(f"{keys}: {values}") 

user_input_2 = input("") 

user_input_2 = int(user_input_2.strip()) 

user_input_product_cat = product_categories_dict.get(user_input_2) 

#print(resulting_matrices["housewares"]) 

Which category is the product in? Enter in number.
1: housewares
2: perfumery
3: auto
4: pet_shop
5: stationery
6: miscellaneous
7: furniture_decor
8: office_furniture
9: garden_tools
10: computers_accessories
11: bed_bath_table
12: toys
13: construction_tools_construction
14: telephony
15: health_beauty
16: electronics
17: baby
18: cool_stuff
19: watches_gifts
20: air_conditioning
21: sports_leisure
22: books_general_interest
23: small_appliances
24: food
25: luggage_accessories
26: fashion_underwear_beach
27: christmas_supplies
28: fashion_bags_accessories
29: musical_instruments
30: construction_tools_lights
31: books_technical
32: costruction_tools_garden
33: home_appliances
34: market_place
35: agro_industry_and_commerce
36: party_supplies
37: home_confort
38: cds_dvds_musicals
39: industry_commerce_and_business
40: consoles_games
41: furniture_bedroom
42: construction_tools_safety
43: fixed_telephony
44: drinks
45: kitchen_dining_laundry_garden_furniture
46: fashion_shoes
47: hom

 1
